In [7]:
import pandas as pd
import numpy as np

In [8]:
# Carregando o dataset que contem o nome dos estados
df_estados = pd.read_csv('../data/temp-brazil/automatic_stations_codes_2000_2021.csv', sep=';')
df_estados.head(2)

,REGIAO,UF,ESTACAO,CODIGO,LATITUDE,LONGITUDE,ALTITUDE
0,N,PA,SANTA MARIA DAS BARREIRAS,A256,-8.729722,-49.856389,165.0
1,SE,SP,CRIOSFERA,C891,-84.000000,-79.494167,1285.0


In [9]:
# Dropando as colunas que não serão utilizadas
df_estados = df_estados.drop(columns=['REGIAO', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'ESTACAO'])
df_estados.head()

,UF,CODIGO
0,PA,A256
1,SP,C891
2,DF,A001
3,BA,A401
4,AM,A101


In [6]:
columns_to_load = ['DATA (YYYY-MM-DD)', 'HORA (UTC)', 'TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C)', 'TEMPERATURA MINIMA NA HORA ANT. (AUT) (C)', 'ESTACAO']
df = pd.read_csv('../data/temp-brazil/automatic_weather_stations_inmet_brazil_2000_2021.csv', sep=';',usecols=columns_to_load)
df.head(2)

,ESTACAO,DATA (YYYY-MM-DD),HORA (UTC),TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C),TEMPERATURA MINIMA NA HORA ANT. (AUT) (C)
0,A001,2021-01-01,0,22.4,20.8
1,A001,2021-01-01,100,21.1,19.5


In [11]:
# Jutando os dois datasets onde o codigo do df_estados é igual a estacao do df, colocando a uf no df
df_merged = df.merge(df_estados, left_on='ESTACAO', right_on='CODIGO', how='left')

In [12]:
# Dropando as colunas que não serão utilizadas
df_merged = df_merged.drop(columns=['CODIGO', 'ESTACAO'])

In [13]:
df_merged.head(2)

,DATA (YYYY-MM-DD),HORA (UTC),TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C),TEMPERATURA MINIMA NA HORA ANT. (AUT) (C),UF
0,2021-01-01,0,22.4,20.8,DF
1,2021-01-01,100,21.1,19.5,DF


In [14]:
# Verificando os tipos de dados
df_merged.dtypes

DATA (YYYY-MM-DD)                             object
HORA (UTC)                                     int64
TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C)    float64
TEMPERATURA MINIMA NA HORA ANT. (AUT) (C)    float64
UF                                            object
dtype: object

In [15]:
# Alterando os dtypes de data para datetime e uf para string
df_merged['DATA (YYYY-MM-DD)'] = pd.to_datetime(df_merged['DATA (YYYY-MM-DD)'])
df_merged['UF'] = df_merged['UF'].astype(str)

In [16]:
df_merged.dtypes

DATA (YYYY-MM-DD)                            datetime64[ns]
HORA (UTC)                                            int64
TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C)           float64
TEMPERATURA MINIMA NA HORA ANT. (AUT) (C)           float64
UF                                                   object
dtype: object

In [29]:
df_average = df_merged.copy()
df_average.head()

,DATA (YYYY-MM-DD),HORA (UTC),TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C),TEMPERATURA MINIMA NA HORA ANT. (AUT) (C),UF
0,2021-01-01,0,22.4,20.8,DF
1,2021-01-01,100,21.1,19.5,DF
2,2021-01-01,200,21.4,19.6,DF
3,2021-01-01,300,20.1,19.3,DF
4,2021-01-01,400,19.4,18.6,DF


In [30]:
# Criando uma coluna com a média anual da temperatura para cada estado
df_average['mean'] = df_average.groupby(['UF', df_average['DATA (YYYY-MM-DD)'].dt.year])['TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C)'].transform('mean')

In [34]:
# Criando a coluna ano a partir da coluna data
df_average['ano'] = df_average['DATA (YYYY-MM-DD)'].dt.year

#Dropando duplicados
df_average = df_average.drop_duplicates(subset=['UF', 'ano'])

In [36]:
# Dropando colunas desnecessarias
df_average = df_average.drop(columns=['DATA (YYYY-MM-DD)', 'HORA (UTC)', 'TEMPERATURA MAXIMA NA HORA ANT. (AUT) (C)', 'TEMPERATURA MINIMA NA HORA ANT. (AUT) (C)'])

In [37]:
# Verificando dados faltantes]
df_average.isnull().sum()

UF      0
mean    0
ano     0
dtype: int64

In [38]:
# Verificando os dados duplicados
df_average.duplicated().sum()

0

In [ ]:
# Dropando os dados duplicados
df_average = df_average.drop_duplicates()

In [41]:
#verificando os anos unicos
df_average['ano'].unique()

array([2021, 2020, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [ ]:
# ordenando os dados por ano
df_average = df_average.sort_values(by='ano')


In [43]:
# Deixando somente dois numeros apos o ponto na media
df_average['mean'] = df_average['mean'].apply(lambda x: round(x, 2))

#alterando o nome da coluna mean para media
df_average = df_average.rename(columns={'mean': 'media'})

df_average.head()

,UF,media,ano
0,DF,23.22,2021
3720,GO,25.23,2021
23064,MS,25.89,2021
56544,MT,25.77,2021
84072,AL,27.49,2021


In [44]:
# Salvando em um csv
df_average.to_csv('../data/prepareted-data/average_temp_brazil.csv', index=False)